In [1]:
from tqdm.notebook import tqdm
from collections import Counter

In [2]:
def read_input(path):
    with open(path) as file:
        lines = [[d for d in line.strip()] for line in file]
    return lines

In [ ]:
test = read_input("test/test_12.txt")
test

[['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'F', 'F'],
 ['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'C', 'F'],
 ['V', 'V', 'R', 'R', 'R', 'C', 'C', 'F', 'F', 'F'],
 ['V', 'V', 'R', 'C', 'C', 'C', 'J', 'F', 'F', 'F'],
 ['V', 'V', 'V', 'V', 'C', 'J', 'J', 'C', 'F', 'E'],
 ['V', 'V', 'I', 'V', 'C', 'C', 'J', 'J', 'E', 'E'],
 ['V', 'V', 'I', 'I', 'I', 'C', 'J', 'J', 'E', 'E'],
 ['M', 'I', 'I', 'I', 'I', 'I', 'J', 'J', 'E', 'E'],
 ['M', 'I', 'I', 'I', 'S', 'I', 'J', 'E', 'E', 'E'],
 ['M', 'M', 'M', 'I', 'S', 'S', 'J', 'E', 'E', 'E']]

In [4]:
input = read_input("input/input_12.txt")

In [5]:
def neighbours(location, len_x, len_y):
    x, y = location
    north = (x, y - 1) if y > 0 else None
    east = (x + 1, y) if x < (len_x - 1) else None
    south = (x, y + 1) if y < (len_y - 1) else None
    west = (x - 1, y) if x > 0 else None
    return [north, east, south, west]

In [6]:
def fence_region(plot, farm_map):
    
    crop = farm_map[plot[1]][plot[0]]
    visited = []
    to_visit = [tuple(plot)]
    perimeter = 0

    while len(to_visit) > 0:
        current_plot = to_visit.pop()
        plot_neighbours = neighbours(current_plot, len(farm_map[0]), len(farm_map))
        for n in plot_neighbours:
            pass
            if n is None: # edge of map
                perimeter += 1
            elif n in visited:
                continue
            elif farm_map[n[1]][n[0]] == crop:
                if n not in to_visit:
                    to_visit.append(n)
            else: # different crop square
                perimeter +=1
        visited.append(current_plot)

    return visited, perimeter, crop

In [7]:
def q12a(farm_map):

    visited = []
    price = 0

    for y, row in enumerate(farm_map):
        for x, plot in enumerate(row):
            if (x, y) not in visited:
                region, perimeter, crop = fence_region((x, y), farm_map)
                visited.extend(region)
                price += len(region) * perimeter
    
    return price

In [8]:
q12a(test)

1930

In [9]:
q12a(input)

1471452

In [19]:
def get_crop(location, farm_map):
    if location is None:
        return ""
    else:
        return farm_map[location[1]][location[0]]

In [21]:
def count_corners(location, farm_map):
    len_x, len_y = len(farm_map[0]), len(farm_map)
    crop = get_crop(location, farm_map)
    x, y = location
    n, e, s, w = neighbours(location, len_x, len_y)
    ne = (x + 1, y - 1) if x < (len_x - 1) and y > 0 else None
    se = (x + 1, y + 1) if x < (len_x - 1) and y < (len_y - 1) else None
    sw = (x - 1, y + 1) if x > 0 and y < (len_y - 1) else None
    nw = (x - 1, y - 1) if x > 0 and y > 0 else None
    corners = 0
    #convex corners
    if get_crop(n, farm_map) != crop and get_crop(w, farm_map) != crop: corners += 1
    if get_crop(n, farm_map) != crop and get_crop(e, farm_map) != crop: corners += 1
    if get_crop(s, farm_map) != crop and get_crop(w, farm_map) != crop: corners += 1
    if get_crop(s, farm_map) != crop and get_crop(e, farm_map) != crop: corners += 1
        
    # concave corners
    if get_crop(n, farm_map) == crop and get_crop(w, farm_map) == crop and get_crop(nw, farm_map) != crop: corners += 1
    if get_crop(n, farm_map) == crop and get_crop(e, farm_map) == crop and get_crop(ne, farm_map) != crop: corners += 1
    if get_crop(s, farm_map) == crop and get_crop(w, farm_map) == crop and get_crop(sw, farm_map) != crop: corners += 1
    if get_crop(s, farm_map) == crop and get_crop(e, farm_map) == crop and get_crop(se, farm_map) != crop: corners += 1

    return corners

In [29]:
def sides_region(plot, farm_map):
    
    crop = get_crop(plot, farm_map)
    visited = []
    to_visit = [tuple(plot)]
    sides = 0

    while len(to_visit) > 0:
        current_plot = to_visit.pop()
        plot_neighbours = neighbours(current_plot, len(farm_map[0]), len(farm_map))
        for n in plot_neighbours:
            if n in visited:
                continue
            elif get_crop(n, farm_map) == crop:
                if n not in to_visit:
                    to_visit.append(n)
        sides += count_corners(current_plot, farm_map)
        visited.append(current_plot)

    return visited, sides, crop
sides_region((0, 2), test)

([(0, 2),
  (0, 3),
  (0, 4),
  (0, 5),
  (0, 6),
  (1, 6),
  (1, 5),
  (1, 4),
  (2, 4),
  (3, 4),
  (3, 5),
  (1, 3),
  (1, 2)],
 10,
 'V')

In [30]:
def q12b(farm_map):

    visited = []
    price = 0

    for y, row in enumerate(farm_map):
        for x, plot in enumerate(row):
            if (x, y) not in visited:
                region, sides, crop = sides_region((x, y), farm_map)
                visited.extend(region)
                price += len(region) * sides
    
    return price
q12b(test)

1206

In [32]:
q12b(input)

863366